### Parse `Husky`

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../')
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import r2rpy
import math
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


/home/rilab/.pyenv/versions/3.8.13/envs/llm/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Parse `scene_husky.xml`

In [2]:
xml_path = '../../asset/husky/scene_husky_w_plate.xml'
env = MuJoCoParserClass(name='Husky',rel_xml_path=xml_path,VERBOSE=True)
print ("[%s] parsed."%(env.name))

dt:[0.0020] HZ:[500]
n_dof (=nv):[10]
n_geom:[11]
geom_names:['floor', None, None, None, None, None, None, None, None, None, None]
n_body:[6]
body_names:['world', 'base_husky', 'front_left_wheel_link', 'front_right_wheel_link', 'rear_left_wheel_link', 'rear_right_wheel_link']
n_joint:[5]
joint_names:['base_husky_joint', 'front_left_wheel', 'front_right_wheel', 'rear_left_wheel', 'rear_right_wheel']
joint_types:[0 3 3 3 3]
joint_ranges:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
n_rev_joint:[4]
rev_joint_idxs:[1 2 3 4]
rev_joint_names:['front_left_wheel', 'front_right_wheel', 'rear_left_wheel', 'rear_right_wheel']
rev_joint_mins:[0. 0. 0. 0.]
rev_joint_maxs:[0. 0. 0. 0.]
rev_joint_ranges:[0. 0. 0. 0.]
n_pri_joint:[0]
pri_joint_idxs:[]
pri_joint_names:[]
pri_joint_mins:[]
pri_joint_maxs:[]
pri_joint_ranges:[]
n_ctrl:[4]
ctrl_names:['front_left_actuator', 'front_right_actuator', 'rear_left_actuator', 'rear_right_actuator']
ctrl_joint_idxs:[7, 8, 9, 10]
ctrl_joint_names:['front_left_whe

### `MPPI`: Model Predictive Path Integral

In [3]:
import numpy as np

# Constants for the robot and control
WHEEL_WIDTH = 0.2854 * 2  # Distance between the two wheels
WHEEL_RADIUS = 0.17775 / 2.0
DT = 0.1  # Time step
HORIZON = 15  # Planning horizon
NUM_SAMPLES = 25  # Number of random control samples

# def differential_drive_model(x, u):
#     """
#     Differential drive robot dynamics model.

#     Parameters:
#         x: numpy.array, state of the robot [x, y, theta].
#         u: numpy.array, control input [v, w].

#     Returns:
#         numpy.array, new state after applying control input.
#     """
#     v, w = u

#     # compute wheel speeds for forward motion
#     vl = v - (w * WHEEL_WIDTH / 2)
#     vr = v + (w * WHEEL_WIDTH / 2)

#     ctrl = [vl,vr,vl,vr]
#     # Update the simulation state
#     env.step(ctrl, ctrl_idxs=[0,1,2,3])

#     p_base = env.get_p_body('base_husky')
#     R_base = r2rpy(env.get_R_body('base_husky'))
#     x_new = p_base[0]
#     y_new = p_base[1]
#     theta_new = R_base[2]

#     return np.array([x_new, y_new, theta_new])

def differential_drive_model(x, u):
    """
    Differential drive robot dynamics model.

    Parameters:
        x: numpy.array, state of the robot [x, y, theta].
        u: numpy.array, control input [v, w].

    Returns:
        numpy.array, new state after applying control input.
    """
    v, w = u
    x_new = x[0] + v * np.cos(x[2]) * DT
    y_new = x[1] + v * np.sin(x[2]) * DT
    theta_new = x[2] + w * DT
    return np.array([x_new, y_new, theta_new])

def cost_function(x_goal, x):
    """
    Cost function for the MPPI controller.

    Parameters:
        x_goal: numpy.array, goal state [x_goal, y_goal, theta_goal].
        x: numpy.array, current state [x, y, theta].

    Returns:
        float, cost value.
    """
    # You can choose any cost function that suits your problem.
    # For example, a simple cost function could be the squared distance to the goal.
    return np.sum((x_goal[:2] - x[:2])**2)

def random_control_samples():
    """
    Generate random control samples.

    Returns:
        numpy.array, random control samples of shape (NUM_SAMPLES, 2).
    """
    # Generate random control inputs within desired ranges.
    v_samples = np.random.uniform(low=-1.0, high=1.0, size=NUM_SAMPLES)
    w_samples = np.random.uniform(low=-1.0, high=1.0, size=NUM_SAMPLES)
    return np.stack((v_samples, w_samples), axis=-1)

def mppi_control(x_init, x_goal):
    """
    MPPI controller for the differential drive mobile robot.

    Parameters:
        x_init: numpy.array, initial state [x, y, theta].
        x_goal: numpy.array, goal state [x_goal, y_goal, theta_goal].

    Returns:
        numpy.array, optimal control input [v_opt, w_opt].
    """
    x_curr = np.copy(x_init)

    for t in range(HORIZON):
        # Generate random control samples
        u_samples = random_control_samples()

        # Calculate the predicted states for each sample
        x_samples = np.zeros((NUM_SAMPLES, 3))
        for i in range(NUM_SAMPLES):
            x_samples[i] = differential_drive_model(x_curr, u_samples[i])

        # Compute the cost for each sample
        costs = np.array([cost_function(x_goal, x_s) for x_s in x_samples])

        # Exponentially weighted sum of costs
        beta = 3  # A parameter that affects the exploration-exploitation trade-off
        weights = np.exp(-beta * costs)
        weights /= np.sum(weights)

        # Optimal control is the weighted average of control samples
        v_opt = np.sum(weights * u_samples[:, 0])
        w_opt = np.sum(weights * u_samples[:, 1])

        # Apply the optimal control to the robot and update the state
        x_curr = differential_drive_model(x_curr, [v_opt, w_opt])

    return np.array([v_opt, w_opt])

### Render

#### Target position tracking

In [9]:
env.init_viewer(viewer_title='Husky',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=107.08,distance=2.5,elevation=-27,lookat=[0.4,0.3,0.2],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.1,contactheight=0.1,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])

husky_base_pos_list = []
wheel_radius = 0.17775 / 2.0
wheel_base = 0.2854 * 2
# Define the velocity gains
v_max = 30.0  # maximum linear velocity, in m/s
w_max = 300.0  # maximum angular velocity, in rad/s

p_target = np.array([5, -1.5, 0.5])

vx_lst, w_lst = [], []
env.reset()
while (env.get_sim_time() < 100.0) and env.is_viewer_alive():

    # Get the current position of the vehicle from the simulation
    p_base = env.get_p_body('base_husky')

    # Calculate the distance to the target point
    d_pos = p_target[:2] - p_base[:2]
    distance = np.linalg.norm(d_pos)
    if distance < 0.1:
        break

    v_mppi, w_mppi = mppi_control(p_base, p_target)
    # scale the linear and angular velocities
    v_mppi = v_max * np.tanh(v_mppi)
    w_mppi = w_max * np.tanh(w_mppi)
    print(f"Optimal control input: [v, w] =[{v_mppi}, {w_mppi}]")

    # compute wheel speeds for forward motion
    vl = v_mppi - (w_mppi * wheel_base / 2)
    vr = v_mppi + (w_mppi * wheel_base / 2)

    ctrl = [vl,vr,vl,vr]
    # Update the simulation state
    env.step(ctrl, ctrl_idxs=[0,1,2,3])

    # Do render
    if env.loop_every(HZ=30) or (env.tick == 1):
        # Update viewer information
        env.plot_sphere(p=p_target,r=0.1,rgba=[1,1,1,1],label='Target position')
        env.plot_T(p=p_base+np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label='[%.2f]sec'%(env.get_sim_time()))
        
        husky_base_pos_list.append(env.get_p_body('base_husky'))
        for p in np.array(husky_base_pos_list)[::5]:
            env.plot_sphere(p=p+np.array([0,0,0.3]), r=0.01) 
        env.render()
    
# Close viewer
env.close_viewer()
print ("Time:[%.2f]sec. Done."%(env.get_sim_time()))



Optimal control input: [v, w] =[12.035081574572352, -52.06035798856534]
Optimal control input: [v, w] =[17.546216687295257, 1.0195973236215248]
Optimal control input: [v, w] =[18.580628239723826, 2.496750605370833]
Optimal control input: [v, w] =[13.922517524246611, 24.7459582616023]
Optimal control input: [v, w] =[10.99943571486803, -27.236688829533144]
Optimal control input: [v, w] =[13.137794290548936, 33.180807402182765]
Optimal control input: [v, w] =[16.892018466629846, -86.89343339657931]
Optimal control input: [v, w] =[18.550464297993752, -37.94023783764225]
Optimal control input: [v, w] =[17.192482602721945, 38.92287978557606]
Optimal control input: [v, w] =[17.063835807275375, -42.545962629282066]
Optimal control input: [v, w] =[14.666672081045983, 108.79194497329823]
Optimal control input: [v, w] =[16.636063404788285, 65.37331827027357]
Optimal control input: [v, w] =[11.965374326713311, -26.565328953332937]
Optimal control input: [v, w] =[15.120490028976171, -103.6588524791